In [15]:
import sklearn.mixture
import pickle
import numpy as np

In [3]:
AMFCC_path='/home/ubuntu/genderRecognition/AMFCC.pk'
GMM_Path='/home/ubuntu/genderRecognition/GMM.pk'
numc_comp=256

In [34]:
male_model= sklearn.mixture.GaussianMixture(n_components=numc_comp, 
                                       covariance_type='full')
female_model=sklearn.mixture.GaussianMixture(n_components=numc_comp, 
                                       covariance_type='full')

In [4]:
f=open(AMFCC_path,'rb')
[train_set_AMFCC,valid_set_AMFCC,test_set_AMFCC,train_set_y,valid_set_y,test_set_y]=pickle.load(f)
f.close()

In [30]:
train_female_feats=np.concatenate([train_set_AMFCC[i] for i in range(len(train_set_AMFCC))[:len(train_set_AMFCC)/2]],axis=0)
train_male_feats=np.concatenate([train_set_AMFCC[i] for i in range(len(train_set_AMFCC))[len(train_set_AMFCC)/2:]],axis=0)

In [31]:
train_female_feats.shape

(4162576, 39)

In [32]:
train_male_feats.shape

(4450915, 39)

In [ ]:
male_model.fit(train_male_feats)

In [ ]:
female_model.fit(train_female_feats)

In [ ]:
f=open(GMM_Path,'wb')
pickle.dump((male_model,female_model), f)
f.close()

In [ ]:
def GenderPredict(feats,mode='score'):
    male=0
    male_score_total=0
    female=0
    female_score_total=0
    for i in range(len(feats)):
        male_score=male_model.score(feats[i])
        female_score=female_model.score(feats[i])
        if male_score>female_score:
            male_score_total+=male_score
            male+=1
        elif male_score<female_score:
            female_score_total+=female_score
            female+=1
    if mode=='score':
        if male_score_total>female_score_total:
            return 1
        else:
            return 0     
    if mode=='most':
        if male>female:
            return 1
        else:
            return 0     
    

In [ ]:
prediction=[]
for i in range(len(test_set_y)):
    prediction.append(GenderPredict(test_set_AMFCC[i]))
comparison=[[prediction[i]==test_set_y[i] for i in range(len(test_set_AMFCC))]]
accuracy=sum()
    